# Fitting of multivariate Gaussian on the spot size dataset

## load the data

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import re
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import curve_fit

from scipy.stats import kde
from scipy import optimize

from matplotlib.ticker import NullFormatter
from matplotlib import pyplot, transforms
import matplotlib
from matplotlib.ticker import AutoMinorLocator
%matplotlib inline
import os
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

pd.set_option("display.max_columns", 300)


fname = '/Users/hkromer/02_PhD/02_Data/01_COMSOL/01_IonOptics/02.current_chamber/02.MR/temporal_refinement/data//02.MR.102.temporal_refinement.002.particleData.csv'
figname = 'COMSOL_ion_beam_multivariate_fit'
outfolder = '/Users/hkromer/polybox/Thesis/Chapters/DetailedNeutronGeneratorCharacterization/Figures/COMSOL_ion_beam_multivariate_fit'

data = pd.read_csv(fname, skiprows=7, index_col=0)
cols = [c for c in data.columns if "2.5E" in c]
data = data[cols]
new_cols = []
for c in cols:
    if 'qx' in c:
        new_cols.append('qx')
    if 'qy' in c:
        new_cols.append('qy')
    if 'qz' in c:
        new_cols.append('qz')
data.columns = new_cols
nbins = 200
lim = 3
#     print(data[pd.isnull(data).any(axis=1)])
y = data['qy'].values
z = data['qz'].values
X = np.asarray([y,z])
np.shape(X)[0]

In [ ]:
def compute_mu(X):
    """
    m: number of samples
    X: vector with the all datapoints, in this case the y and z position. numpy array with (number of features, samples)
    """
    # number of features, in this case it is 2, because y and z position of datapoint
    num_features = np.shape(X)[0]
    # number of samples
    m = np.shape(X)[1]
    # initialize mu
    mu = np.zeros(num_features)
    mu = (1/m) * np.sum(X, axis=1) # y, z
    
    return mu

def compute_sig(X, mu):
    """
    Computes variance of the multivariate Gaussian.
    """
    # number of features, in this case it is 2, because y and z position of datapoint
    num_features = np.shape(X)[0]
    # number of samples
    m = np.shape(X)[1]  
    sig = np.zeros(num_features)

    t_mu = np.tile(mu, reps=(m,1)).T # tile mu to have same shape as X

    sig = (1/m) * np.dot((X-t_mu), (X-t_mu).T) # y, z

    return sig

# multivariate Gaussian
# def gaussian(X, )
mu = compute_mu(X)
sig = compute_sig(X, mu)
mu, sig

In [ ]:
def gaussian(x, X):
    """
    X is the training set
    x is the query set (one new example)
    """
    mu = compute_mu(X)
    sig = compute_sig(X, mu)
    
    # number of features
    n = X.shape[0]
    
    # determinant of sigma
    det_sig = np.linalg.det(sig)
    
    # return probability
    # p = (1/(2pi)^n/2 det^0.5) exp(-0.5 * (x-mu)^T sig^-1 (x-mu))
    # p = A exp(B)
    A = 1 / (np.power(2*np.pi,(n/2)) * np.power(det_sig,0.5))
    B = -0.5 * np.dot( np.dot( (x-mu),np.linalg.inv(sig) ), (x-mu) ) 

    prob = A * np.exp(B)
    
    return prob
    
x = np.asarray([-2,-2])
gaussian(x, X)

In [ ]:
nbins = 500
lim = 3
xi, yi = np.mgrid[-lim:lim:nbins*1j, -lim:lim:nbins*1j]
# values to query, all the datapoints
values = np.vstack([xi.flatten(), yi.flatten()])
zi = np.asarray([gaussian(val, X) for val in values.T])

In [ ]:
# plot
fs = 20

f = plt.figure(1, figsize=(9, 9))

nullfmt = NullFormatter()         # no labels
# definitions for the axes
left, width = 0.12, 0.65
bottom, height = 0.12, 0.65
rect_scatter = [left, bottom, width, height]
axScatter = plt.axes(rect_scatter)
p = axScatter.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='gouraud', cmap=plt.cm.jet)
plt.axis('equal')
plt.xlabel(r'\textbf{y [mm]}', fontsize=fs)
plt.ylabel(r'\textbf{z [mm]}', fontsize=fs)
axScatter.tick_params('x', colors='black', labelsize=fs-2)
axScatter.tick_params('y', colors='black', labelsize=fs-2)
plt.yticks(np.arange(-3,4,1))
plt.xticks(np.arange(-3,4,1))


left = 0.8
bottom = 0.12
width = 0.05
height = 0.65
cax = f.add_axes([left, bottom, width, height])
cbar = f.colorbar(p, cax)
# cbar = f.colorbar(p, cax, ticks=[0,0.05,0.1,0.15,0.3])
print(np.max(zi))
cbar.ax.tick_params(labelsize=fs-2)
plt.savefig(f'{outfolder}/{figname}.pdf')
plt.tight_layout()
plt.show()

In [ ]:
# plot along y=0
xi = np.linspace(-3,3,1000)
yi = np.zeros(xi.shape[0])
values = np.vstack([yi.flatten(), xi.flatten()])

zi = np.asarray([gaussian(val, X) for val in values.T]).reshape(xi.shape)

# find closest value to the maximum
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx, array[idx]

idx_max, max_val = find_nearest(zi, np.max(zi))
# maximum value along the y axis in the 2D plot: this is the centerpoint through which the z plot should go!
max_X = xi[idx_max]
print(f"Maximum value: idx: {idx_max}, max_val: {max_val}")
print(f"Maximum along x: idx: {idx_max}, max_val: {max_X}")

# find FWHM
print(f'Maximum value: {np.max(zi)}')
# left side (z < 0)
zi_left = zi[0:idx_max]
idx_FWHM_left, val_at_idx_half_max_val = find_nearest(zi_left, np.max(zi)/2)
print(f"Half max at left: idx: {idx_FWHM_left}, value: {val_at_idx_half_max_val}")

# right side
zi_right = zi[idx_max:]
idx_FWHM_right, val_at_idx_half_max_val = find_nearest(zi_right, np.max(zi)/2)
idx_FWHM_right = idx_FWHM_right + idx_max
print(f"Half max at right: idx: {idx_FWHM_right}, value: {val_at_idx_half_max_val}")
print([ xi[idx_FWHM_left], xi[idx_FWHM_right] ])
FWHM = xi[idx_FWHM_left] - xi[idx_FWHM_right]
print(f"FWHM: {FWHM}")

# top plot
f = plt.figure(1, figsize=(8, 6.4))
ylims = (-0.01, 0.3)
# top plot
plt.subplot(2, 1, 1)
plt.plot(xi, zi, color='darkorange', linewidth=2.0, label='y=0')
# plt.plot(qry_eval, Y_fit_x, '-.',color='darkred',linewidth=2.0, label='Gaussian fit')
plt.plot( [ xi[idx_FWHM_right], xi[idx_FWHM_left] ], [ val_at_idx_half_max_val, val_at_idx_half_max_val ], '--', color='black' )
plt.text(-0.3, .09, r'\textbf{FWHM = 1.96 mm}', fontsize=14)
plt.xlabel(r'\textbf{z [mm]}', fontsize=fs)
plt.ylabel(r'\textbf{Estimated PDF [-]}', fontsize=fs)
ax = plt.gca()
ax.tick_params('x', colors='black', labelsize=fs-2)
ax.tick_params('y', colors='black', labelsize=fs-2)
minor_locator = AutoMinorLocator(2)
ax.xaxis.set_minor_locator(minor_locator)
minor_locator = AutoMinorLocator(2)
ax.yaxis.set_minor_locator(minor_locator)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
# plt.yticks(np.arange(0,0.3,0.1))
plt.xticks(np.arange(-3,4,1))
plt.xlim(-3,3)
leg1 = plt.legend(loc="best",  fontsize=12)
plt.ylim(ylims)
plt.tight_layout()


# plot along z=0
xi = np.linspace(-3,3,300)
yi = np.ones(xi.shape[0]) * max_X
values = np.vstack([xi.flatten(), yi.flatten()])

zi = np.asarray([gaussian(val, X) for val in values.T]).reshape(xi.shape)

idx_max, max_val = find_nearest(zi, np.max(zi))

# find FWHM
print(f'Maximum value: {np.max(zi)}')
# left side (z < 0)
zi_left = zi[0:idx_max]
idx_FWHM_left, val_at_idx_half_max_val = find_nearest(zi_left, np.max(zi)/2)
print(f"Half max at left: idx: {idx_FWHM_left}, value: {val_at_idx_half_max_val}")

# right side
zi_right = zi[idx_max:]
idx_FWHM_right, val_at_idx_half_max_val = find_nearest(zi_right, np.max(zi)/2)
idx_FWHM_right = idx_FWHM_right + idx_max
print(f"Half max at right: idx: {idx_FWHM_right}, value: {val_at_idx_half_max_val}")
print([ xi[idx_FWHM_left], xi[idx_FWHM_right] ])
FWHM = xi[idx_FWHM_left] - xi[idx_FWHM_right]
print(f"FWHM: {FWHM}")


ylims = (-0.01, 0.3)
# top plot
plt.subplot(2, 1, 2)
plt.plot(xi, zi, color='darkred', linewidth=2.0, label=f'z={max_X:.2f}mm')
# plt.plot(qry_eval, Y_fit_x, '-.',color='darkred',linewidth=2.0, label='Gaussian fit')
plt.plot( [ xi[idx_FWHM_right], xi[idx_FWHM_left] ], [ val_at_idx_half_max_val, val_at_idx_half_max_val ], '--', color='black' )
plt.text(-0.8, .08, r'\textbf{FWHM = 1.71 mm}', fontsize=14)
plt.xlabel(r'\textbf{y [mm]}', fontsize=fs)
plt.ylabel(r'\textbf{Estimated PDF [-]}', fontsize=fs)
ax = plt.gca()
ax.tick_params('x', colors='black', labelsize=fs-2)
ax.tick_params('y', colors='black', labelsize=fs-2)
minor_locator = AutoMinorLocator(2)
ax.xaxis.set_minor_locator(minor_locator)
minor_locator = AutoMinorLocator(2)
ax.yaxis.set_minor_locator(minor_locator)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
plt.xticks(np.arange(-3,4,1))
plt.xlim(-3,3)
plt.xticks(np.arange(-3,4,1))
leg1 = plt.legend(loc="best",  fontsize=12)
plt.ylim(ylims)
plt.tight_layout()
plt.savefig(f'{outfolder}/{figname}_along_yz.pdf')

print(f'Maximum value: {np.max(zi)}')
